<a class="anchor" id="1-bullet">     
    

## 1. Importing Data & Libraries 

</a>


In [10]:
! pip show squarify

Name: scikit-learn
Version: 1.0.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\carolinashaul\anaconda3\lib\site-packages
Requires: threadpoolctl, joblib, scipy, numpy
Required-by: tslearn, scikit-learn-intelex, mlxtend, imbalanced-learn


In [1]:
#Basic Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import confusion_matrix
import squarify

#To find addresses
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

#Plot Maps
import folium
from folium.plugins import HeatMap, MarkerCluster 

#Functions
import Functions

%load_ext autoreload
%autoreload 2

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
individuals = pd.read_csv('clientes_preprocessed.csv')
transactions = pd.read_csv('Customer Basket Dataset.csv')
prod_mapping = pd.read_excel('Product Mapping Excel File.xlsx')

In [3]:
individuals.set_index('customer_id', inplace=True)

<br>

---
<br>

<a class="anchor" id="2.4.-bullet">     
    

### 2.4. Data Adjustments

</a>

#### Joining Features

In [4]:
individuals['lifetime_spend_animal_protein'] = individuals[['lifetime_spend_fish', 'lifetime_spend_meat']].sum(axis=1)
individuals['lifetime_spend_technology'] = individuals[['lifetime_spend_videogames', 'lifetime_spend_electronics']].sum(axis=1)

In [5]:
corr_matrix = individuals.drop(['lifetime_spend_fish', 'lifetime_spend_meat', 'lifetime_spend_videogames', 'lifetime_spend_electronics'], axis = 1).corr()
Functions.get_high_correlations(corr_matrix, 0.66)

,Variable 1,Variable 2,Correlation
10,lifetime_total_distinct_products,lifetime_spend_animal_protein,0.932928
3,lifetime_spend_groceries,lifetime_total_distinct_products,0.926119
2,distinct_stores_visited,percentage_of_products_bought_promotion,0.923021
6,lifetime_spend_groceries,lifetime_spend_animal_protein,0.915666
5,lifetime_spend_groceries,total_lifetime_spend,0.913175
0,kids_home,minors_in_household,0.895671
11,total_lifetime_spend,lifetime_spend_animal_protein,0.872337
9,lifetime_total_distinct_products,total_lifetime_spend,0.855891
1,teens_home,minors_in_household,0.799848
7,lifetime_spend_nonalcohol_drinks,lifetime_spend_technology,0.787628


In [6]:
continuous = ['lifetime_spend_groceries', 'lifetime_spend_electronics', 'lifetime_spend_vegetables',
       'lifetime_spend_nonalcohol_drinks', 'lifetime_spend_alcohol_drinks',
       'lifetime_spend_meat', 'lifetime_spend_fish', 'lifetime_spend_hygiene',
       'lifetime_spend_videogames', 'lifetime_total_distinct_products',
       'percentage_of_products_bought_promotion', 'year_first_transaction', 'total_lifetime_spend', 'age', 'birthday' ]

#Variaveis tais como 'lifetime_total_distinct_products', 'year_first_transaction', 
#  'age', 'birthday', estão a ser tratadas como continuas neste contexto dada a amplitude delas.

discrete = ['kids_home', 'teens_home', 'typical_hour', 'birthmonth',
       'number_complaints', 'distinct_stores_visited', 'minors_in_household', 'education']

binary = ['customer_gender', 'loyalty_card']

#### Scalling

In [7]:
scaler = MinMaxScaler()
individuals_num = individuals.drop(['customer_name', 'region'], axis=1)
scaled_data = scaler.fit_transform(individuals_num)
scaled_data = pd.DataFrame(scaled_data, index= individuals_num.index, columns=individuals_num.columns)

## hierarchical

In [75]:
hierarchical = AgglomerativeClustering(
    linkage='ward', distance_threshold=0, n_clusters=None
    ).fit(scaled_data)

In [76]:
from scipy.cluster.hierarchy import dendrogram

In [77]:
def plot_dendrogram(model, **kwargs):
    '''
    Create linkage matrix and then plot the dendrogram
    Arguments: 
    - model(HierarchicalClustering Model): hierarchical clustering model.
    - **kwargs
    Returns:
    None, but dendrogram plot is produced.
    '''
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)


In [ ]:
fig, ax = plt.subplots()
plt.title("hierarchical clustering dendrogram")
plot_dendrogram(hierarchical, truncate_mode="level", p = 50)
plt.show()

In [ ]:
individuals['agg_ward'] = AgglomerativeClustering(
    linkage='ward', n_clusters=6
    ).fit_predict(scaled_data)

In [ ]:
Functions.compare_clusters(individuals, 'agg_ward')

## DBScan

In [68]:
from sklearn.cluster import DBSCAN

### Sol1

In [106]:
len(individuals)

29774

In [113]:
individuals['dbscan_clustering1'] = DBSCAN(
    eps=0.65, min_samples=175
    ).fit_predict(scaled_data)

In [114]:
Functions.compare_clusters(individuals, 'dbscan_clustering1')

,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,general_mean
customer_gender,0.522370,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.496574
kids_home,1.148730,1.329577,0.681818,1.172564,1.148839,0.994406,0.996499,1.001336,1.301882,0.701522,0.987408,0.999148,1.015740,2.372340,0.692946,2.328904,0.650000,1.155404
teens_home,0.567110,0.800368,0.067708,0.782438,0.765589,0.273427,0.271008,0.072128,0.816407,0.070201,0.265477,0.068172,0.275971,1.450000,0.049793,1.282392,0.081818,0.516827
number_complaints,0.695284,0.608096,0.512311,0.318480,0.307237,1.549650,1.554622,1.018700,0.616779,0.487482,1.555089,1.007669,1.543547,0.736170,0.535270,0.681063,0.459091,0.770169
distinct_stores_visited,9.178960,2.565777,2.001420,2.007864,1.995904,8.011888,7.975490,19.998664,2.547990,1.988218,8.022036,20.003835,7.994753,3.024468,1.958506,2.990033,1.986364,6.159636
lifetime_spend_groceries,855.389359,3525.303588,200.362216,997.479249,998.502959,14986.167133,15018.905462,300.734194,3486.052754,200.626411,14987.550892,305.014061,14971.423924,5430.981915,198.846473,5533.922481,200.500000,4008.813629
lifetime_spend_electronics,154.561064,433.542318,4999.624053,50.175186,49.916705,199.810490,199.857843,20.000000,440.281664,5002.638193,200.336831,20.000000,200.012592,380.995745,4987.062241,414.590255,4984.081818,972.556727
typical_hour,10.891173,17.303588,20.942235,9.971603,9.991807,11.896503,11.906863,9.000000,17.251220,20.941581,11.985310,8.986792,11.882476,17.622340,20.917012,17.821705,21.177273,14.461107
lifetime_spend_vegetables,648.002418,390.956072,20.060133,1499.970293,1499.632681,796.779720,805.231793,299.939003,387.359517,20.008837,802.651626,300.999574,792.141658,558.124468,19.933610,541.353267,20.118182,568.474340
lifetime_spend_nonalcohol_drinks,248.908102,528.937213,1501.721117,19.934906,20.110150,899.841958,900.771709,301.375779,522.585406,1500.325970,902.028332,300.156370,900.982162,786.384043,1500.933610,760.301218,1499.277273,633.505206


### Sol2

In [146]:
individuals['dbscan_clustering2'] = DBSCAN(
    eps=0.65, min_samples=175
    ).fit_predict(scaled_data[columns2])

In [147]:
Functions.compare_clusters(individuals, 'dbscan_clustering2')

,0,1,2,3,4,5,general_mean
customer_gender,0.498680,0.490962,0.488984,0.497350,0.522777,0.540650,0.496574
kids_home,1.233562,0.691492,1.000214,1.494406,0.672451,1.000000,1.155404
teens_home,0.735345,0.068932,0.070160,0.704416,0.065076,0.065041,0.516827
number_complaints,0.642065,0.500121,1.013262,1.163297,0.498915,0.959350,0.770169
distinct_stores_visited,3.076908,1.994939,20.001283,5.725810,1.971800,20.012195,6.159636
lifetime_spend_groceries,4197.198310,200.491926,303.180963,10473.899706,199.635575,298.756098,4008.813629
lifetime_spend_electronics,292.563771,5001.103880,20.000000,270.611973,4985.639913,20.000000,972.556727
typical_hour,14.414312,20.941914,8.991658,14.010010,21.041215,8.991870,14.461107
lifetime_spend_vegetables,769.456166,20.034948,300.456898,726.248675,20.021692,295.955285,568.474340
lifetime_spend_nonalcohol_drinks,423.136256,1501.036153,300.826096,795.785280,1500.143167,292.939024,633.505206


### Sol3

In [148]:
individuals['dbscan_clustering3'] = DBSCAN(
    eps=0.65, min_samples=200
    ).fit_predict(scaled_data[columns2])

In [149]:
Functions.compare_clusters(individuals, 'dbscan_clustering3')

,0,1,2,3,4,5,general_mean
customer_gender,0.498680,0.490962,0.488984,0.497350,0.522777,0.540650,0.496574
kids_home,1.233562,0.691492,1.000214,1.494406,0.672451,1.000000,1.155404
teens_home,0.735345,0.068932,0.070160,0.704416,0.065076,0.065041,0.516827
number_complaints,0.642065,0.500121,1.013262,1.163297,0.498915,0.959350,0.770169
distinct_stores_visited,3.076908,1.994939,20.001283,5.725810,1.971800,20.012195,6.159636
lifetime_spend_groceries,4197.198310,200.491926,303.180963,10473.899706,199.635575,298.756098,4008.813629
lifetime_spend_electronics,292.563771,5001.103880,20.000000,270.611973,4985.639913,20.000000,972.556727
typical_hour,14.414312,20.941914,8.991658,14.010010,21.041215,8.991870,14.461107
lifetime_spend_vegetables,769.456166,20.034948,300.456898,726.248675,20.021692,295.955285,568.474340
lifetime_spend_nonalcohol_drinks,423.136256,1501.036153,300.826096,795.785280,1500.143167,292.939024,633.505206
